# VacationPy
---

In [1]:
# dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
from pprint import pprint

# import Geoapify API key
from api_keys import geoapify_key

In [2]:
# load in city data csv from WeatherPy
city_data_df = pd.read_csv('../output_data/cities_data.csv').set_index("City_ID")

# display city data df
city_data_df.head()

,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
City_ID,,,,,,,,,
0,puerto natales,CL,1731445654,-51.7236,-72.4875,17.25,51,75,11.83
1,jamestown,US,1731445580,42.0970,-79.2353,7.23,57,0,4.12
2,la'ie,US,1731445654,21.6477,-157.9253,27.81,75,91,4.02
3,beloyarskiy,RU,1731445654,63.7119,66.6722,-21.45,100,39,1.53
4,waitangi,NZ,1731445654,-43.9535,-176.5597,13.23,71,9,2.68


### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city
---

In [3]:
# configure map plot
humidity_map = city_data_df.hvplot.points("Lng", "Lat", geo=True, tiles="OSM", size="Humidity", 
                                      color="City", frame_width = 700, frame_height = 500, alpha=0.7)

# display map
humidity_map

c:\Users\brian\Anaconda3\Lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated and will be removed in a future release
  "class": algorithms.Blowfish,


:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# create new df based on ideal factors: max temp is 23-33C; cloudiness is 26-60%; wind speed is 3-9 m/s
ideal_weather_df = city_data_df[((city_data_df['Max Temp']>= 23) & (city_data_df['Max Temp']<= 33)) & 
                                ((city_data_df['Cloudiness']>=26) & (city_data_df['Cloudiness']<= 60)) &
                                ((city_data_df['Wind Speed']>=3) & (city_data_df['Wind Speed']<= 9))]

# display new df
ideal_weather_df

,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
City_ID,,,,,,,,,
20,enewetak,MH,1731445659,11.3474,162.3373,28.44,77,31,6.12
25,ta`u,AS,1731445660,-14.2336,-169.5144,27.19,81,52,8.88
46,arraial do cabo,BR,1731445665,-22.9661,-42.0278,24.92,83,40,6.69
69,namibe,AO,1731445671,-15.1961,12.1522,23.91,63,35,3.65
83,jabor,MH,1731445675,5.9210,169.6434,28.23,75,51,4.10
121,avarua,CK,1731445684,-21.2078,-159.7750,24.03,64,40,6.69
216,sechura,PE,1731445187,-5.5569,-80.8222,24.01,59,30,7.28
233,san antonio de pale,GQ,1731445209,-1.4014,5.6325,26.33,84,51,6.77
253,toliara,MG,1731445232,-23.3500,43.6667,24.54,64,43,3.44


### Step 3: Create a new DataFrame called `hotel_df`

In [5]:
# in new df store city, country, coordinates, and humidity
hotel_df = ideal_weather_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# add new blank column, Hotel Name
hotel_df['Hotel Name'] = ''

# display new df
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
City_ID,,,,,,
20,enewetak,MH,11.3474,162.3373,77,
25,ta`u,AS,-14.2336,-169.5144,81,
46,arraial do cabo,BR,-22.9661,-42.0278,83,
69,namibe,AO,-15.1961,12.1522,63,
83,jabor,MH,5.9210,169.6434,75,
121,avarua,CK,-21.2078,-159.7750,64,
216,sechura,PE,-5.5569,-80.8222,59,
233,san antonio de pale,GQ,-1.4014,5.6325,84,
253,toliara,MG,-23.3500,43.6667,64,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates

In [14]:
# setup query url
base_url = "https://api.geoapify.com/v2/places?"

# setup intial parameters for query (first hotel within 10,000 m)
radius = 10000
limit = 1

# setup parameters dictionary
params_dict = {
    "apiKey": geoapify_key,
    "categories":"accommodation.hotel",
    "limit": limit
}

In [15]:
# print message to indicate start hotel search
print("---------------------")
print("Starting hotel search")
print("---------------------")

# iterate through the hotel_df
for index, row in hotel_df.iterrows():

    # add coordinates parameters to params dictionary
    params_dict['filter'] = f"circle:{row['Lng']},{row['Lat']},{radius}"
    params_dict['bias'] = f"proximity:{row['Lng']},{row['Lat']}"
    
    # make api request
    hotel_data = requests.get(base_url, params=params_dict).json()

    # grab first hotel name from the results and store in hotel_df
    try:
        # add hotel name
        hotel_df.loc[index, 'Hotel Name'] = hotel_data['features'][0]['properties']['name']
    except (IndexError, KeyError):
        # add NA if hotel not found
        hotel_df.loc[index, 'Hotel Name'] = "NA"
    
    # log results for each city
    print(f"{row['City'].title()}--> {row['Hotel Name']}")

# print message to indicate end of hotel search
print("----------------------")
print("Completed hotel search")
print("----------------------")

# display completed hotel_df
hotel_df

---------------------
Starting hotel search
---------------------
Enewetak--> NA
Ta`U--> NA
Arraial Do Cabo--> NA
Namibe--> Nelsal Pensao
Jabor--> NA
Avarua--> Paradise Inn
Sechura--> Maximus Inn
San Antonio De Pale--> NA
Toliara--> Ambary
Afaahiti--> Omati Lodge
Brisas De Zicatela--> Casa de Olas
Mahina--> Motu Martin
Funafuti--> Vaiaku Lagi Hotel
Buchanan--> Phillipmena's Guesthouse
Primero De Mayo--> NA
Kavieng--> Kavieng Hotel
Progreso--> Centro Vacacional Obrero CTM
Khorixas--> iGowati Country Hotel and Lodge
----------------------
Completed hotel search
----------------------


,City,Country,Lat,Lng,Humidity,Hotel Name
City_ID,,,,,,
20,enewetak,MH,11.3474,162.3373,77,NA
25,ta`u,AS,-14.2336,-169.5144,81,NA
46,arraial do cabo,BR,-22.9661,-42.0278,83,NA
69,namibe,AO,-15.1961,12.1522,63,Nelsal Pensao
83,jabor,MH,5.9210,169.6434,75,NA
121,avarua,CK,-21.2078,-159.7750,64,Paradise Inn
216,sechura,PE,-5.5569,-80.8222,59,Maximus Inn
233,san antonio de pale,GQ,-1.4014,5.6325,84,NA
253,toliara,MG,-23.3500,43.6667,64,Ambary
